In [5]:
import talib
from talib import abstract
import pandas as pd
import yfinance as yf

In [7]:
# 取得 TALIB 支援的所有技術指標名稱列表
all_ta_indicators = talib.get_functions()
# 列出 TALIB 支援的技術指標總量
print(f"TALIB 支援的技術指標總量: {len(all_ta_indicators)}")

TALIB 支援的技術指標總量: 158


In [9]:
# 取得依類別分組後的技術指標名稱列表，例如分成動量指標、Cycle指標等
all_group_ta_indicators = talib.get_function_groups()
# 列出所有技術指標分類的名稱
print(f"TALIB 所有技術指標分類的名稱: {list(all_group_ta_indicators)}")

TALIB 所有技術指標分類的名稱: ['Cycle Indicators', 'Math Operators', 'Math Transform', 'Momentum Indicators', 'Overlap Studies', 'Pattern Recognition', 'Price Transform', 'Statistic Functions', 'Volatility Indicators', 'Volume Indicators']


In [10]:
print(len(all_group_ta_indicators))

10


In [11]:
# 使用 yfinance 下載台泥（股票代號 1101.TW）從 2022-01-01 到 2022-12-31 的股市資料
data = (
    pd.DataFrame(yf.download("1101.TW", start="2022-01-01", end="2022-12-31"))
    .droplevel("Ticker", axis=1)
    .reset_index()
    .ffill()
)

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_14766/4108826518.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  pd.DataFrame(yf.download("1101.TW", start="2022-01-01", end="2022-12-31"))
[*********************100%***********************]  1 of 1 completed


In [12]:
# 方法ㄧ：使用標準 TALIB 函數進行技術指標計算
# 計算 30 天的簡單移動平均線(SMA)，並將結果存入新的欄位
data["SMA"] = talib.SMA(real=data["Close"], timeperiod=30)

# 計算 14 天的相對強弱指標 (RSI)，並將結果存入新的欄位 "RSI"
data["RSI"] = talib.RSI(data["Close"], timeperiod=14)

# 查看隨機指標 (STOCH)，的參數和使用說明
print("STOCH 函數說明：")
help(talib.STOCH)

# 計算隨機指標 (STOCH)，包括%K線和%D線，並將結果存入對應欄位
# %K 和 %D 是隨機指標的兩條線，快線 (fast %K) 和慢線 (slow %D)
data["STOCH_K"], data["STOCH_D"] = talib.STOCH(
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    fastk_period=5, # 快線 %K 的週期為 5
    slowk_period=3, # 慢線 %D 的週期為 3
    slowk_matype=0, # 慢線 %K 的移動平均類型為簡單移動平均
    slowd_period=3, # 慢線 %D 的週期為 3
    slowd_matype=0, # 慢線 %D 的移動平均類型為簡單移動平均
)



STOCH 函數說明：
Help on function STOCH in module talib._ta_lib:

STOCH(
    high,
    low,
    close,
    fastk_period=-2147483648,
    slowk_period=-2147483648,
    slowk_matype=0,
    slowd_period=-2147483648,
    slowd_matype=0
)
    STOCH(ndarray high, ndarray low, ndarray close, int fastk_period=-0x80000000, int slowk_period=-0x80000000, int slowk_matype=0, int slowd_period=-0x80000000, int slowd_matype=0)

    STOCH(high, low, close[, fastk_period=?, slowk_period=?, slowk_matype=?, slowd_period=?, slowd_matype=?])

    Stochastic (Momentum Indicators)

    Inputs:
        prices: ['high', 'low', 'close']
    Parameters:
        fastk_period: 5
        slowk_period: 3
        slowk_matype: 0
        slowd_period: 3
        slowd_matype: 0
    Outputs:
        slowk
        slowd



In [13]:
data

Price,Date,Close,High,Low,Open,Volume,SMA,RSI,STOCH_K,STOCH_D
0,2022-01-03,38.680428,39.251056,38.598912,39.169537,13591954,NaN,NaN,NaN,NaN
1,2022-01-04,38.558151,38.802708,38.313598,38.721189,12594043,NaN,NaN,NaN,NaN
2,2022-01-05,38.435879,38.558155,38.313602,38.395118,10688966,NaN,NaN,NaN,NaN
3,2022-01-06,38.802711,38.802711,38.435878,38.558155,13608960,NaN,NaN,NaN,NaN
4,2022-01-07,38.680428,38.843465,38.476632,38.802708,12026649,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
241,2022-12-26,31.681591,31.727639,31.405295,31.635539,12280553,30.955554,57.602212,62.851896,62.740689
242,2022-12-27,31.589491,31.957883,31.589491,31.819735,11350887,30.969369,56.480542,56.000000,60.304000
243,2022-12-28,31.589491,31.635539,31.267151,31.451347,13791324,31.012347,56.480542,59.333312,59.395069
244,2022-12-29,31.036907,31.543444,31.036907,31.543444,13374728,30.995463,49.740763,36.666604,50.666639


In [14]:
# 計算移動平均收斂發散指標 (MACD)，包括 MACD 線、訊號線和柱狀圖，並將結果存入對應欄位
data["MACD"], data["MACDSignal"], data["MACDHist"] = talib.MACD(
    data["Close"],
    fastperiod=12, # 快線的週期為 12
    slowperiod=26, # 慢線的週期為 26
    signalperiod=9, # 訊號線的週期為 9
)

In [15]:
# 計算布林帶 (Bollinger Bands)，包括上軌線、中軌線和下軌線，並將結果存入對應欄位
data["BBANDS_upper"], data["BBANDS_middle"], data["BBANDS_lower"] = talib.BBANDS(
    data["Close"],
    timeperiod=5, # 布林帶的週期為 5
    nbdevup=2.0, # 上軌線與中軌線的標準差為 2
    nbdevdn=2.0, # 下軌線與中軌線的標準差為 2
    matype=0, # 移動平均的類型為簡單移動平均
)

In [16]:
# 查看最後幾筆資料
print(data.tail())

Price       Date      Close       High        Low       Open    Volume  \
241   2022-12-26  31.681591  31.727639  31.405295  31.635539  12280553   
242   2022-12-27  31.589491  31.957883  31.589491  31.819735  11350887   
243   2022-12-28  31.589491  31.635539  31.267151  31.451347  13791324   
244   2022-12-29  31.036907  31.543444  31.036907  31.543444  13374728   
245   2022-12-30  30.990860  31.313200  30.990860  31.221104   8488022   

Price        SMA        RSI    STOCH_K    STOCH_D      MACD  MACDSignal  \
241    30.955554  57.602212  62.851896  62.740689  0.296890    0.267385   
242    30.969369  56.480542  56.000000  60.304000  0.299076    0.273724   
243    31.012347  56.480542  59.333312  59.395069  0.297380    0.278455   
244    30.995463  49.740763  36.666604  50.666639  0.248582    0.272480   
245    31.006208  49.213748  16.666634  37.555517  0.203843    0.258753   

Price  MACDHist  BBANDS_upper  BBANDS_middle  BBANDS_lower  
241    0.029505     31.936020      31.43292

In [18]:
## 方法二：使用 TALIB 的 abstract 模組進行技術指標計算
# 使用 abstract 模組時，資料欄位必續符合 TALIB 所需的格式
# TALIB 要求的資料欄位名稱為 "open", "high", "low", "close"

# 重新命名資料欄位名稱，使其符合 TALIB 所需的格式
data = data.rename(
    columns={"Open": "open", "High": "high", "Low": "low", "Close": "close"}
)

# 使用 TALIB 的 abstract 模組計算 30 天的簡單移動平均線 (SMA)，並將結果存入 "SMA" 欄位
data["SMA"] = talib.abstract.SMA(data, timeperiod=30)

# 使用 TALIB 的 abstract 模組計算 14 天的相對強弱指標 (RSI)，並將結果存入 "RSI" 欄位
data["RSI"] = talib.abstract.RSI(data, timeperiod=14)

# 查看後幾筆資料
print(data.tail())

Price       Date      close       high        low       open    Volume  \
241   2022-12-26  31.681591  31.727639  31.405295  31.635539  12280553   
242   2022-12-27  31.589491  31.957883  31.589491  31.819735  11350887   
243   2022-12-28  31.589491  31.635539  31.267151  31.451347  13791324   
244   2022-12-29  31.036907  31.543444  31.036907  31.543444  13374728   
245   2022-12-30  30.990860  31.313200  30.990860  31.221104   8488022   

Price        SMA        RSI    STOCH_K    STOCH_D      MACD  MACDSignal  \
241    30.955554  57.602212  62.851896  62.740689  0.296890    0.267385   
242    30.969369  56.480542  56.000000  60.304000  0.299076    0.273724   
243    31.012347  56.480542  59.333312  59.395069  0.297380    0.278455   
244    30.995463  49.740763  36.666604  50.666639  0.248582    0.272480   
245    31.006208  49.213748  16.666634  37.555517  0.203843    0.258753   

Price  MACDHist  BBANDS_upper  BBANDS_middle  BBANDS_lower  
241    0.029505     31.936020      31.43292